In [1]:
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv
import os
from datetime import datetime
import pandas as pd
import json

In [2]:
variable = 'has_residential'
run_number = '3'

directory = "data"
data_file = "housing_descriptions_training.csv"
data_path = directory + "/" + data_file

data_df = pd.read_csv(data_path)
data_df.head()


,short_description,entitlement,proposed_adding,residential_add,adu_udu_add,multi_family_add,single_family_add,has_residential,has_market_rate,has_affordable_lowinc,has_livework,has_adu,has_udu,has_adu_udu,has_multi_family,has_single_family,has_non_res_sqft,has_mixeduse
0,NaN,ADDITIONAL GRADING IN COMPLIANCE WITH AND TO A...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,NaN,DELETE CONDITION S-3(I)(A) OF VTT 71898,49.0,49.0,0.0,49.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,NaN,DEMOLITION OF EXISTING BUILDING TO CREATE TWO ...,75.0,75.0,0.0,75.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,NaN,"INCREASE GRADING, 2 NEW RETAINING WALLS TO COR...",1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NaN,MODIFICATION FROM AN APPROVED 12 UNIT DENSITY ...,12.0,12.0,0.0,12.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [3]:
# Sample 100 rows for training set
train_df = data_df.sample(n=100, random_state=900)

# Sample another 100 rows for test set, ensuring no overlap with train set
test_df = data_df[~data_df.index.isin(train_df.index)].sample(n=40, random_state=900)

# Extract only the specified columns for both sets
train_df = train_df[['short_description', 'entitlement', variable]]
test_df = test_df[['short_description', 'entitlement', variable]]

print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

# Display first few rows of each set
print("\nTraining set sample:")
train_df.head()

Training set shape: (100, 3)
Test set shape: (40, 3)

Training set sample:


,short_description,entitlement,has_residential
2626,"NEW 1,485 SQ FT. SFD WITH ATTACHED GARAGE WITH...","PURSUANT TO LAMC SECTION 11.5.7C, PROJECT PERM...",1.0
2024,DEMOLITION OF AN EXISTING 6-UNIT APT. AND CONS...,"PURSUANT TO LAMC SECTION 11.5.7 C, PROJECT PER...",1.0
181,2 UNIT CONDOMINIUMS,PER LAMC 17.50 TO ALLOW PRELIMINARY PARCEL MAP...,1.0
16,NaN,MODIFY LOT 2 AND LOT 5 OF APPROVED MAP VTT-704...,1.0
2675,NEW 2-STORY 620 SQUARE FEET SINGLE-FAMILY RESI...,"PURSUANT TO LAMC SECTION 11.5.7, PROJECT PERMI...",1.0


In [4]:

#3/29
#HAS_RESIDENTIAL
# instructions = """
# You are a housing assistant specializing in evaluating development projects. Your task is to read the **Entitlement** and **Short Description** fields and determine whether the project includes **any residential component**.

# ### Output:
# - **1** if the project includes residential use of any kind (e.g., single-family homes, apartments, ADUs, mixed-use with residential).
# - **0** if the project does not include any residential use (e.g., entirely commercial, retail, office, industrial).
# - **-1** if it's unclear whether the project has residential use.

# ### Criteria:

# #### **Classify as Residential (1)** if:
# - The project includes **any new residential units** (single-family homes, duplexes, apartments, etc.).
# - It mentions **multifamily**, **single-family**, **housing**, **residences**, **dwelling units**, **ADUs**, **density bonus**, or **live/work units** with residential purpose.
# - It is a **mixed-use** development that includes at least one **residential unit**.

# #### **Classify as Not Residential (0)** if:
# - The project is entirely for **non-residential purposes**, such as:
#   - Commercial, retail, restaurants, bars
#   - Offices, warehouses, or industrial use
#   - Parking structures with no mention of housing
#   - Zone changes, permit adjustments, or compliance work with no mention of housing
# - **Mixed-use** projects where the **residential component is not mentioned at all**.

# ### Example:

# Short Description:
# "A DENSITY BONUS TO ALLOW THE CONSTRUCTION OF A 5-STORY BUILDING WITH 70 RESIDENTIAL UNITS ABOVE GROUND-FLOOR RETAIL."

# Entitlement:
# "PURSUANT TO 12.22.A.25; A DENSITY BONUS FOR A 5-STORY, 70-UNIT SENIOR HOUSING PROJECT WITH AFFORDABLE HOUSING SET-ASIDES."

# Expected Output:
# 1
# """

#Has market rate
# instructions = """
# You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes **any market-rate housing units**.

# ### Output:
# - **1** if the project includes market-rate housing units.
# - **0** if all residential units are explicitly affordable or income-restricted.
# - **-1** if it is unclear whether the project includes any housing units at all.

# ### Criteria:

# #### Output **1** if:
# - The project includes any residential units (e.g., SFDs, ADUs, apartments), and **not all units are labeled as affordable**.
# - Projects with a **mix** of affordable and unspecified units (e.g., “17 of 70 units are low-income”) — the unspecified units are assumed to be market-rate.
# - Projects that include **housing** but do **not mention affordability** at all — assume units are market-rate by default.

# #### Output **0** if:
# - **All** housing units are explicitly labeled as:
#   - Affordable
#   - Low-income
#   - Very low income
#   - Extremely low income
#   - Moderate income
#   - Supportive housing
#   - Income-restricted

# #### Output **-1** if:
# - The description does **not make it clear whether any housing is being built**.
# - The project is purely commercial, institutional, or involves zoning/permits with no mention of new residential units.

# ### Example:

# Short Description:
# "A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

# Entitlement:
# "17 units set aside as low-income; remaining 53 units are market-rate."

# Expected Output:
# 1
# """

#HAS MARKET RATE 3
# instructions = """
# You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes **any market-rate housing units**.

# ###Definitions:
# ### Definitions:

# **UNITS** include any of the following:
# - Dwelling units, apartments, condos, houses, residential units, hotel rooms (if residential), SFD (single-family dwelling), ADU, JADU, duplex, triplex, fourplex, live/work units, and small-lot subdivisions.

# **LOWINC** refers to explicitly income-restricted housing, including:
# - Affordable
# - Low-income
# - Very low income
# - Extremely low income
# - Moderate income
# - Supportive housing
# - Income-restricted


# ### Output:
# - **0** if all units are explicitly affordable or income-restricted.
# - **1** if otherwise, not all UNITS are explicitly affordable 

# ### Criteria:

# #### Output **0** if:
# - **All** UNITS are explicitly labeled as LOWINC

# #### Output **1** if:
# - The project includes any UNITS, and **not all UNITS are labeled as affordable**.
# - Projects with a **mix** of affordable and unspecified UNITS (e.g., “17 of 70 units are low-income”) — the unspecified units are assumed to be market-rate.
# - Projects that include **housing** but do **not mention affordability** at all — assume UNITS are market-rate by default.

# ### Example:

# Short Description:
# "A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

# Entitlement:
# "17 units set aside as low-income; remaining 53 units are market-rate."

# Expected Output:
# 1
# """

#HAS MARKET RATE 4
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes **any market-rate housing units**.

---

### Definitions:

**UNITS** include any of the following:
- Dwelling units, apartments, condos, houses, residential units, hotel rooms (if residential), SFD (single-family dwelling), ADU, JADU, duplex, triplex, fourplex, live/work units, and small-lot subdivisions.

**LOWINC** refers to explicitly income-restricted housing, including:
- Affordable
- Low-income
- Very low income
- Extremely low income
- Moderate income
- Supportive housing
- Income-restricted

**Secondary or non-market-rate UNITS** include:
- ADUs or JADUs attached to an existing single-family home
- Changes of use or legalization of previously unpermitted units
- Projects where no **new primary residential development** is occurring

---

### Output:
- **0** if all UNITS are explicitly LOWINC, or if only secondary units (e.g., ADUs) or legalized/unpermitted units are involved.
- **1** if any UNITS are not explicitly LOWINC and are **new primary residential development**.

---

### Criteria:

#### Output **0** if:
- **All** residential UNITS are explicitly LOWINC.
- The only units are **ADUs/JADUs** attached to an existing SFD with no affordability mention.
- The only units are **unpermitted**, legalized, or changes of use — and there is no clear new construction or affordability data.

#### Output **1** if:
- The project includes any UNITS and **not all** are LOWINC.
- The project includes a **mix** of affordable and unspecified units (e.g., “17 of 70 units are low-income”) — the unspecified are assumed market-rate.
- The project involves **new residential construction** and affordability is not mentioned — assume market-rate.

---

### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
1
"""

#Has affordable 1
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes **any affordable or income-restricted housing units**.

---

### Definitions:

**LOWINC** refers to explicitly income-restricted housing, including:
- Affordable housing
- Low-income, very low income, extremely low income
- Moderate income
- Supportive housing
- Income-restricted units
- Units required by a Density Bonus, Mello Act, or other affordability programs

**UNITS** include:
- Dwelling units, apartments, condos, houses, residential units, hotel rooms (if residential), SFD, ADU, JADU, duplex, triplex, fourplex, live/work units, small-lot subdivisions

---

### Output:

- **1** if the project includes any units that are explicitly income-restricted or affordable
- **0** if there is no mention of any income-restricted, affordable, or low-income housing

---

### Criteria:

#### Output **1** if:
- The project includes units labeled as:
  - “Affordable”
  - “Low-income”
  - “Very low income”
  - “Moderate income”
  - “Supportive housing”
  - “Set aside for income-restricted households”
  - Or any reference to affordability programs (e.g., “density bonus with restricted units,” “subject to Mello Act affordability”)

#### Output **0** if:
- Residential units are included, but there is **no mention of affordability** or income restriction
- The units are market-rate by implication (e.g., new SFDs, ADUs, apartments, condos without affordability language)
- The only activity is legalization or change-of-use (e.g., “legalize unpermitted unit”)

#### Output **-1** if:
- It is unclear whether any residential units are involved (e.g., project is entirely commercial, zoning, signage, or grading)

# ### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."
Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
1
"""

#live work 0
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes **live/work units**.

### Output:
- **1** if the project explicitly mentions live/work units.
- **0** if otherwise.

### Criteria:

#### Output **1** if:
- The text contains **explicit phrases**, such as:
  - “live/work unit(s)”
  - “live-work space”
  - “live/work loft(s)”
  - “living/working quarters”

#### Output **0** if:
- There is **no explicit mention** of live/work units, even if the project is mixed-use, commercial-residential, or includes flexible space.

### Example:

Short Description: "New 4-story building with 10 live/work units"
Entitlement: "Includes 10 live/work lofts"
→ Expected Output: **1**

Short Description: "New apartment building with ground-floor retail"
Entitlement: "Mixed-use project with 20 residential units and 5,000 sf commercial space"
→ Expected Output: **0**
"""

#has adu 0 
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes an **Accessory Dwelling Unit (ADU)**.

### Output:
- **1** if the project includes an ADU.
- **0** if not.

### Criteria:

#### Output **1** if:
- The text explicitly mentions:
  - “ADU”
  - “Accessory Dwelling Unit”
  - “JADU” (Junior Accessory Dwelling Unit)
- The ADU can be new, attached, detached, or part of a remodel.

#### Output **0** if:
- There is no mention of ADU, Accessory Dwelling Unit, or JADU in the text.

### Examples:

**Example A**
Short Description: "New single-family home with detached ADU"
Entitlement: "Includes a 600 sq ft detached Accessory Dwelling Unit"
→ Expected Output: **1**

**Example B**
Short Description: "New SFD with garage and basement"
Entitlement: "No mention of ADU"
→ Expected Output: **0**
"""

#has udu 0
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes an **Urban Dwelling Unit (UDU)**.

### Output:
- **1** if the project includes a UDU.
- **0** if not.

### Criteria:

#### Output **1** if:
- The text explicitly mentions:
  - “UDU”
  - “Urban Dwelling Unit”

#### Output **0** if:
- There is no mention of UDU or Urban Dwelling Unit in the text.

### Examples:

**Example A**
Short Description: "Construct new UDU at rear of lot"
Entitlement: "1 Urban Dwelling Unit (UDU) behind existing structure"
→ Expected Output: **1**

**Example B**
Short Description: "New apartment building with 12 units"
Entitlement: "No mention of UDU"
→ Expected Output: **0**
"""

#has non res 0
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes **any non-residential or commercial use**, such as retail, office, restaurant, industrial, or other commercial components.

### Output:
- **1** if the project includes any commercial or non-residential use.
- **0** if it is entirely residential.

### Criteria:

#### Output **1** if:
- The text mentions any of the following:
  - “mixed-use”
  - “commercial use”
  - “retail”
  - “office”
  - “restaurant”
  - “industrial”
  - “warehouse”
  - “storefront” or “shop”
- Any non-residential use, even without specifying square footage

#### Output **0** if:
- The project is **entirely residential** (e.g., single-family homes, ADUs, apartments) and no commercial or non-residential use is mentioned

### Examples:

**Example A**
Short Description: "Mixed-use building with 10 apartments above ground-floor retail"
Entitlement: "Ground-floor commercial use"
→ Expected Output: **1**

**Example B**
Short Description: "New duplex with garage"
Entitlement: "Two residential units"
→ Expected Output: **0**
"""

#has mixed use
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project is **mixed-use**, meaning it includes **both residential and non-residential components**.

### Output:
- **1** if the project includes both residential and non-residential use, or explicitly says “mixed-use.”
- **0** if not.

### Criteria:

#### Output **1** if:
- The text **explicitly says “mixed-use”**
- Or, the project includes **both**:
  - Residential use (e.g., apartments, housing units, SFDs, ADUs)
  - **And** non-residential use (e.g., commercial, office, retail, restaurant, industrial)

#### Output **0** if:
- The project is only residential
- Or only non-residential
- Or residential use is mentioned, but the only other features are garages, pools, or parking (not commercial)

### Examples:

**Example A**
Short Description: "Mixed-use building with 30 apartments and 5,000 sq ft of retail"
Entitlement: "Includes residential units and ground-floor commercial"
→ Expected Output: **1**

**Example B**
Short Description: "New 12-unit apartment building"
Entitlement: "No commercial use"
→ Expected Output: **0**

**Example C**
Short Description: "New SFD with detached garage"
Entitlement: "Residential only"
→ Expected Output: **0**
"""

#market rate 5
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. 

---

### Definitions:

**UNITS** include any of the following:
- Dwelling units, apartments, condos, houses, residential units, hotel rooms (if residential), SFD (single-family dwelling), ADU, JADU, duplex, triplex, fourplex, live/work units, and small-lot subdivisions.

**LOWINC** refers to explicitly income-restricted housing, including:
- Affordable
- Low-income
- Very low income
- Extremely low income
- Moderate income
- Supportive housing
- Income-restricted

**Secondary or non-market-rate UNITS** include:
- ADUs or JADUs **attached to an existing** single-family home
- Legalizations of previously unpermitted units **without new construction**
- Changes of use (e.g. converting space into housing) **without significant new development or additions**
- Projects where no **new primary residential development** is occurring

---

### Output:
- **0** if all UNITS are explicitly LOWINC.
- **1** if any UNITS are not explicitly LOWINC and are **new primary residential development**.

---

### Criteria:

#### Output **0** if:
- **All**  UNITS are explicitly LOWINC.

#### Output **1** if:
- The project includes any new UNITS and **not all** are explicitly LOWINC.
- The project includes a **mix** of affordable and unspecified units (e.g., “17 of 70 units are low-income”) — the unspecified units are assumed to be market-rate.
- The project involves **new construction** of any residential UNITS (e.g. new SFD, new duplex, new apartment), and there is **no affordability designation** — assume market-rate.
- The project includes **an SFD with an ADU/JADU** and the **SFD is newly constructed** — assume market-rate.

---

### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
1
"""

#market 6
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields.

---

### Definitions:

**UNITS** include: dwelling units, apartments, condos, houses, residential units, hotel rooms (if residential), SFDs (single-family dwellings), ADUs, JADUs, duplexes, triplexes, fourplexes, live/work units, and small-lot subdivisions.

**LOWINC** units are explicitly income-restricted housing, such as:
- Affordable
- Low-income
- Very low income
- Extremely low income
- Moderate income
- Supportive housing
- Income-restricted

---

### Output:
- Output **0** if *all* new or existing UNITS are income-restricted, or if the project involves **no net change** to housing quantity or type.
- Output **1** if **any** of the following are true: new units are created or enabled, affordability is partial or missing, or units are altered in a way that increases market value.

---

### Output **0** if:
- All new or all existing UNITS are explicitly LOWINC.
- The only action is **legalizing** an already existing, previously unpermitted *dwelling unit*, with **no new structure or change in use**.

---

### Output **1** if:
- Any **new unit** is created: including ADUs, duplexes, or conversion of non-housing space (e.g., garage, rec room, basement) into a dwelling.
- A **unit is rebuilt**, enlarged, or significantly upgraded (e.g., demo and rebuild of SFD, or manufactured home replacement).
- An **existing unit is converted into another use** (e.g., to condos, bed & breakfast, or short-term rental).
- A **density bonus** or other incentive is used and *not all units* are LOWINC.
- A **zone change**, tract map, subdivision, or other **land entitlement** is requested and **no affordability is mentioned**.
- **Affordability is partial** (e.g., “2 of 26 units are affordable”) — remaining units are assumed market-rate.
- The language indicates **intent to redevelop, intensify, or commercialize housing** (e.g., demolition, change of use, upzoning).

---

### Assumptions:
- Assume **market-rate intent** unless it is clearly stated that **100% of units are LOWINC**.
- **Adding, enlarging, converting, or replacing** units implies market-rate housing unless explicitly affordable.

---

### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
1
"""

#adu 1
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields. Your task is to determine whether the project includes an **Accessory Dwelling Unit (ADU)**.

---

### Output:
- **1** if the project includes an ADU.
- **0** if not.

---

### Output **1** if:
- The text explicitly mentions:
  - “ADU”
  - “Accessory Dwelling Unit”
  - “JADU” (Junior ADU)
- OR if there is strong indirect evidence:
  - “Garage conversion” (only if a unit is being added or repurposed for living space)
  - “Recreation room” or “Rec room” being added, legalized, or modified with a garage
  - “Second-story addition” to a garage or detached structure
  - “Non-conforming addition” with use or structure implying habitable space
  - Zoning language around “Accessory use”, “reduced side yard” + garage additions in residential zones

Only infer ADUs from **indirect language** when it strongly suggests a residential or second unit being added or legalized.

---

### Output **0** if:
- There is no mention of “ADU”, “Accessory Dwelling Unit”, or “JADU”
- The project involves a garage, recreation room, or addition, but **not** as living space or housing
- The text discusses subdivision, zoning, or construction of new homes **without** referencing an ADU

---

### Example:

Short Description: “Conversion of garage and rec room into habitable space”
Entitlement: “Zoning adjustment for reduced setbacks and recreation use with side yard exception”
→ Output: 1 (this likely implies an ADU)
"""

#mixed use 2
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields.

---

### Shared Definitions:

**Residential Uses** include:
- Apartments, condos, dwelling units, single-family homes (SFD), ADUs, JADUs, duplexes, triplexes, fourplexes, live/work units (if no separate commercial), small-lot subdivisions

**Non-Residential Uses** include:
- Commercial, retail, restaurant, bar, office, administrative office, medical, industrial, warehouse
- Institutional uses: community center, school, religious facility

**Mixed-Use** means a project includes **both** residential and non-residential uses (see above)

**Live/Work Units** are considered **residential** unless explicitly stated that they include **separate or public-facing commercial space**

**Common Misleading Phrases**:
- "FAR", "Density Bonus", "Height Increase", and "Open Space Waiver" do **not** imply mixed-use by themselves.
- Garages, basements, storage, and parking are **not** considered non-residential uses.

### Task:
Determine whether the project is **mixed-use**, meaning it includes **both residential and non-residential** uses.

---

### Output:
- Output **1** if the project includes both residential and non-residential components (or explicitly says "mixed-use")
- Output **0** if not

---

### Output **1** if:
- Residential and non-residential uses are both present
- Example: “60 apartments and 5,000 sq ft of ground floor commercial space”
- Example: “Community center with a new residential unit”

### Output **0** if:
- The project is only residential (even if it has garages, basements, or parking)
- The project is only non-residential
- The project includes live/work units, but no separate or public-facing commercial space is described

---
### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
0
"""



#residential use 2
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields.

---

### Shared Definitions:

**Residential Uses** include:
- Apartments, condos, dwelling units, single-family homes (SFD), ADUs, JADUs, duplexes, triplexes, fourplexes, live/work units (if no separate commercial), small-lot subdivisions

**Non-Residential Uses** include:
- Commercial, retail, restaurant, bar, office, administrative office, medical, industrial, warehouse
- Institutional uses: community center, school, religious facility

**Mixed-Use** means a project includes **both** residential and non-residential uses (see above)

**Live/Work Units** are considered **residential** unless explicitly stated that they include **separate or public-facing commercial space**

**Common Misleading Phrases**:
- "FAR", "Density Bonus", "Height Increase", and "Open Space Waiver" do **not** imply mixed-use by themselves.
- Garages, basements, storage, and parking are **not** considered non-residential uses.

### Task:
Determine whether the project includes **any residential use**.

---

### Output:
- Output **1** if the project includes any residential use (e.g. housing units, apartments, SFDs, ADUs)
- Output **0** if there is no residential use

---

### Output **1** if:
- Any of the following appear: apartments, housing units, condos, homes, duplex, triplex, fourplex, ADU, JADU, live/work units (unless explicitly non-residential)
- Even if only one residential unit is included (e.g., "1 apartment above a retail store")

### Output **0** if:
- No residential use is mentioned
- The project only includes commercial, office, industrial, or institutional use without any housing


---
### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
1
"""

#residential 3
instructions = """
You are a housing assistant. Read the Entitlement and Short Description fields.

---

### Shared Definitions:

**UNITS** include any of the following:
- Dwelling units, apartments, condos, houses, residential units, hotel rooms (if residential), SFD (single-family dwelling), ADU, JADU, duplex, triplex, fourplex, live/work units, and small-lot subdivisions.

**LOWINC** refers to explicitly income-restricted housing, including:
- Affordable
- Low-income
- Very low income
- Extremely low income
- Moderate income
- Supportive housing
- Income-restricted

**Non-Residential Uses** include:
- Commercial, retail, restaurant, bar, office, administrative office, medical, industrial, warehouse
- Institutional uses: community center, school, religious facility

**Mixed-Use** means a project includes **both** residential and non-residential uses (see above)

**Live/Work Units** are considered **residential** unless explicitly stated that they include **separate or public-facing commercial space**

**Common Misleading Phrases**:
- "FAR", "Density Bonus", "Height Increase", and "Open Space Waiver" do **not** imply mixed-use by themselves.
- Garages, basements, storage, and parking are **not** considered non-residential uses.

### Task:
Determine whether the project includes **any residential use**.

---

### Output:
- Output **1** if the project includes any residential UNITS use (e.g. residential such as housing units, apartments, SFDs, ADUs)
- Output **0** if there is no residential use

---

### Output **1** if:
- Any of the following of UNITS appears (UNITS counts as residential)
- Any of the LOWINC appears (LOWINC counts as residential)
- Tract map
- Even if only one residential unit is included (e.g., "1 apartment above a retail store")

### Output **0** if:
- No UNITS use is mentioned
- The project only includes **Non-Residential Uses** such as commercial, office, industrial, or institutional use without any housing


---
### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

Expected Output:
1
"""



In [5]:

# Prepare the data for fine-tuning
def prepare_finetune_data(df):
    examples = []
    
    for _, row in df.iterrows():
        # Get the text input (short_description and entitlement)
        short_desc = str(row['short_description']) if not pd.isna(row['short_description']) else ""
        entitlement = str(row['entitlement']) if not pd.isna(row['entitlement']) else ""
        
        # Combine the text inputs
        text = f"Short Description: {short_desc}\nEntitlement: {entitlement}"
        
        # Get the label (proposed_adding)
        label = row[variable]
        
        # Create the example in the required format
        example = {
            "messages": [
                {"role": "system", "content": instructions},
                {"role": "user", "content": text},
                {"role": "assistant", "content": str(label)}
            ]
        }
        
        examples.append(example)
    
    return examples

# Prepare training and test data
train_examples = prepare_finetune_data(train_df)
test_examples = prepare_finetune_data(test_df)

# Create directory if it doesn't exist
os.makedirs('finetune_data', exist_ok=True)

# Export training data
with open(f'finetune_data/train{variable}.jsonl', 'w') as f:
    for example in train_examples:
        f.write(json.dumps(example) + '\n')

# Export test data
with open(f'finetune_data/test{variable}.jsonl', 'w') as f:
    for example in test_examples:
        f.write(json.dumps(example) + '\n')

print(f"Exported {len(train_examples)} training examples and {len(test_examples)} test examples to JSONL files.")



Exported 100 training examples and 40 test examples to JSONL files.


In [6]:
load_dotenv()
key = os.environ.get("OPENAI_API_KEY")

In [7]:
client = OpenAI(api_key=key)


train_output_file_path = f"finetune_data/train{variable}.jsonl"
validation_output_file_path = f"finetune_data/test{variable}.jsonl"

train_file = client.files.create(
  file=open(train_output_file_path, "rb"),
  purpose="fine-tune"
)

valid_file = client.files.create(
  file=open(validation_output_file_path, "rb"),
  purpose="fine-tune"
)

print(f"Training file Info: {train_file}")
print(f"Validation file Info: {valid_file}")

Training file Info: FileObject(id='file-5CEnBpfeY7ndTdBUqSdn6M', bytes=284785, created_at=1743373070, filename='trainhas_residential.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)
Validation file Info: FileObject(id='file-KqdXExXgYxNrVtREQQpMjV', bytes=113441, created_at=1743373070, filename='testhas_residential.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)


In [8]:
model = client.fine_tuning.jobs.create(
  training_file=train_file.id, 
  validation_file=valid_file.id,
  model="gpt-4o-mini-2024-07-18", 
  hyperparameters={
    "n_epochs": 2,
	"batch_size": 8,
	"learning_rate_multiplier": 0.2
  },
  suffix=f"housing_desc_trainset100_{variable}_run{run_number}"  # Added tag to identify this run
)
job_id = model.id
status = model.status

print(f'Fine-tuning model with jobID: {job_id}.')
print(f"Training Response: {model}")
print(f"Training Status: {status}")
print(f"Run tag: housing_desc_run_1")

Fine-tuning model with jobID: ftjob-xBScwfBNclqrUbAGnhNS5eT6.
Training Response: FineTuningJob(id='ftjob-xBScwfBNclqrUbAGnhNS5eT6', created_at=1743373073, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=8, learning_rate_multiplier=0.2, n_epochs=2), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-J2DCfq0j1nXInCHinBoPqK5v', result_files=[], seed=695437327, status='validating_files', trained_tokens=None, training_file='file-5CEnBpfeY7ndTdBUqSdn6M', validation_file='file-KqdXExXgYxNrVtREQQpMjV', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=8, learning_rate_multiplier=0.2, n_epochs=2)), type='supervised'), user_provided_suffix='housing_desc_trainset100_has_residential_run3', metadata=None)
Training Status: validating_files
Run tag: housing_desc_run_1
